# Fish Recognition Model - EfficientNetB0 Version

## Why EfficientNetB0?
1. **Better architecture** - More reliable for transfer learning than MobileNetV3
2. **Proven track record** - Works well for image classification tasks
3. **Similar size** - ~5MB unquantized, similar to MobileNetV3
4. **Better feature learning** - More layers and better compound scaling

## Key Improvements:
1. **EfficientNetB0** instead of MobileNetV3Small
2. **Class weights** - Forces balanced learning across all 4 classes
3. **Better monitoring** - Tests model during training with raw images
4. **No quantization** - Keeps full float32 precision
5. **Proper preprocessing** - Consistent between training and inference

## Setup:
1. Download dataset from: https://www.kaggle.com/datasets/crowww/a-large-scale-fish-dataset
2. Extract to: C:\Users\s62ht\Downloads\Fish_Dataset
3. Install requirements: `pip install tensorflow numpy pillow scikit-learn`
4. Run all cells in order

In [1]:
%pip install scikit-learn

# Import required libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import shutil
import random
from pathlib import Path
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
TensorFlow version: 2.20.0
GPU Available: []


In [2]:
# Configuration
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30
FINETUNE_EPOCHS = 15
CLASSES = ['Gilt-Head Bream', 'Hourse Mackerel', 'Sea Bass', 'Shrimp']

# Dataset paths
DATASET_BASE = r'C:\Users\s62ht\Downloads\Fish_Dataset\Fish_Dataset\Fish_Dataset'
SOURCE_SEABASS = f'{DATASET_BASE}\\Sea Bass\\Sea Bass'
SOURCE_SHRIMP = f'{DATASET_BASE}\\Shrimp\\Shrimp'
SOURCE_GILTHEAD = f'{DATASET_BASE}\\Gilt-Head Bream\\Gilt-Head Bream'
SOURCE_HORSEMACKEREL = f'{DATASET_BASE}\\Hourse Mackerel\\Hourse Mackerel'

# Working directory
WORK_DIR = r'C:\Users\s62ht\Downloads\fish_model_efficientnet'
TRAIN_DIR = f'{WORK_DIR}\\train'
VAL_DIR = f'{WORK_DIR}\\validation'

os.makedirs(WORK_DIR, exist_ok=True)

print("Configuration complete")
print(f"Dataset path: {DATASET_BASE}")
print(f"Output path: {WORK_DIR}")

Configuration complete
Dataset path: C:\Users\s62ht\Downloads\Fish_Dataset\Fish_Dataset\Fish_Dataset
Output path: C:\Users\s62ht\Downloads\fish_model_efficientnet


In [3]:
# Verify dataset exists
print("Checking dataset availability...")
for source, name in [
    (SOURCE_SEABASS, 'Sea Bass'),
    (SOURCE_SHRIMP, 'Shrimp'),
    (SOURCE_GILTHEAD, 'Gilt-Head Bream'),
    (SOURCE_HORSEMACKEREL, 'Hourse Mackerel')
]:
    if os.path.exists(source):
        count = len(os.listdir(source))
        print(f"✓ {name} images found: {count}")
    else:
        print(f"✗ {name} folder not found at {source}")

Checking dataset availability...
✓ Sea Bass images found: 1000
✓ Shrimp images found: 1000
✓ Gilt-Head Bream images found: 1000
✓ Hourse Mackerel images found: 1000


In [4]:
# Create train/validation split (80/20)
def split_dataset(source_dir, class_name, train_ratio=0.8):
    """Split dataset into train and validation sets"""
    if not os.path.exists(source_dir):
        print(f"✗ Error: Source directory not found: {source_dir}")
        return 0, 0
    
    # Create directories
    train_class_dir = os.path.join(TRAIN_DIR, class_name)
    val_class_dir = os.path.join(VAL_DIR, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)
    
    # Get all image files
    all_files = [f for f in os.listdir(source_dir) 
                 if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not all_files:
        print(f"✗ Warning: No image files found in {source_dir}")
        return 0, 0
    
    # Shuffle and split
    random.seed(42)
    random.shuffle(all_files)
    split_idx = int(len(all_files) * train_ratio)
    
    train_files = all_files[:split_idx]
    val_files = all_files[split_idx:]
    
    # Copy files
    for filename in train_files:
        src = os.path.join(source_dir, filename)
        dst = os.path.join(train_class_dir, filename)
        shutil.copy2(src, dst)
    
    for filename in val_files:
        src = os.path.join(source_dir, filename)
        dst = os.path.join(val_class_dir, filename)
        shutil.copy2(src, dst)
    
    print(f"✓ {class_name}: {len(train_files)} train, {len(val_files)} validation")
    return len(train_files), len(val_files)

print("Splitting dataset...")
split_dataset(SOURCE_GILTHEAD, 'Gilt-Head Bream')
split_dataset(SOURCE_HORSEMACKEREL, 'Hourse Mackerel')
split_dataset(SOURCE_SEABASS, 'Sea Bass')
split_dataset(SOURCE_SHRIMP, 'Shrimp')
print("Dataset split complete!")

Splitting dataset...
✓ Gilt-Head Bream: 800 train, 200 validation
✓ Hourse Mackerel: 800 train, 200 validation
✓ Sea Bass: 800 train, 200 validation
✓ Shrimp: 800 train, 200 validation
Dataset split complete!


In [5]:
# Custom preprocessing function that matches Flutter exactly
# This is CRITICAL - must be exactly (pixel / 127.5) - 1.0
def preprocess_for_efficientnet(x):
    """Preprocess image to [-1, 1] range"""
    # x is in range [0, 255]
    return (x / 127.5) - 1.0

# Create data generators with strong augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_for_efficientnet,
    rotation_range=30,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.25,
    zoom_range=0.25,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_for_efficientnet)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

validation_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

print(f"\nClass indices: {train_generator.class_indices}")
print(f"Total training samples: {train_generator.samples}")
print(f"Total validation samples: {validation_generator.samples}")

Found 3200 images belonging to 4 classes.
Found 800 images belonging to 4 classes.

Class indices: {'Gilt-Head Bream': 0, 'Hourse Mackerel': 1, 'Sea Bass': 2, 'Shrimp': 3}
Total training samples: 3200
Total validation samples: 800


In [6]:
# Calculate class weights to handle any imbalance
# This forces the model to pay equal attention to all classes
class_indices = train_generator.class_indices
class_labels = train_generator.classes

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
class_weight_dict = dict(enumerate(class_weights))

print("\nClass weights (to ensure balanced learning):")
for class_name, class_idx in sorted(class_indices.items(), key=lambda x: x[1]):
    print(f"  {class_name}: {class_weight_dict[class_idx]:.3f}")


Class weights (to ensure balanced learning):
  Gilt-Head Bream: 1.000
  Hourse Mackerel: 1.000
  Sea Bass: 1.000
  Shrimp: 1.000


In [7]:
# Build model with EfficientNetB0
def create_model():
    # Load pre-trained EfficientNetB0 (without top layers)
    base_model = EfficientNetB0(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    # Build classification head with strong regularization
    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    
    # Three-layer classifier for better feature learning
    x = layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer
    outputs = layers.Dense(len(CLASSES), activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    return model, base_model

model, base_model = create_model()

# Compile with label smoothing
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

print("Model created successfully")
print(f"Total parameters: {model.count_params():,}")
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 9s 1us/step
Model created successfully
Total parameters: 4,873,767


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,873,767 (18.59 MB)

 Trainable params: 822,404 (3.14 MB)

 Non-trainable params: 4,051,363 (15.45 MB)

In [8]:
# Helper function to test model with raw images during training
def test_model_with_raw_images(model, labels, num_per_class=3):
    """Test model with raw images exactly like Flutter does"""
    
    def preprocess_image_flutter_style(image_path):
        """Preprocess image exactly as Flutter does"""
        img = Image.open(image_path).convert('RGB')
        
        # Center crop to square
        width, height = img.size
        size = min(width, height)
        left = (width - size) // 2
        top = (height - size) // 2
        img = img.crop((left, top, left + size, top + size))
        
        # Resize to 224x224
        img = img.resize((224, 224), Image.BILINEAR)
        
        # Normalize: (pixel / 127.5) - 1.0
        img_array = np.array(img, dtype=np.float32)
        img_array = (img_array / 127.5) - 1.0
        img_array = np.expand_dims(img_array, axis=0)
        
        return img_array
    
    total_correct = 0
    total_tested = 0
    
    print("\n" + "="*60)
    print("Testing model with raw images...")
    print("="*60)
    
    for class_name in labels:
        class_dir = os.path.join(VAL_DIR, class_name)
        if not os.path.exists(class_dir):
            continue
        
        images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:num_per_class]
        
        print(f"\n{class_name}:")
        class_correct = 0
        
        for img_name in images:
            img_path = os.path.join(class_dir, img_name)
            input_data = preprocess_image_flutter_style(img_path)
            
            # Run inference
            output = model.predict(input_data, verbose=0)[0]
            
            predicted_idx = np.argmax(output)
            predicted_label = labels[predicted_idx]
            confidence = output[predicted_idx]
            
            is_correct = predicted_label == class_name
            status = "✓" if is_correct else "✗"
            
            # Show probabilities
            probs_str = ", ".join([f"{labels[i]}: {output[i]*100:.1f}%" for i in range(len(labels))])
            print(f"  {status} {predicted_label} ({confidence*100:.1f}%) [{probs_str}]")
            
            if is_correct:
                class_correct += 1
                total_correct += 1
            total_tested += 1
        
        print(f"  Class accuracy: {class_correct}/{len(images)} = {class_correct/len(images)*100:.1f}%")
    
    accuracy = total_correct / total_tested if total_tested > 0 else 0
    print("\n" + "="*60)
    print(f"OVERALL RAW IMAGE ACCURACY: {total_correct}/{total_tested} = {accuracy*100:.1f}%")
    print("="*60 + "\n")
    
    return accuracy

print("Helper function defined")

Helper function defined


In [9]:
# Custom callback to test with raw images every 5 epochs
class RawImageTestCallback(keras.callbacks.Callback):
    def __init__(self, labels, test_frequency=5):
        super().__init__()
        self.labels = labels
        self.test_frequency = test_frequency
    
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.test_frequency == 0:
            test_model_with_raw_images(self.model, self.labels, num_per_class=3)

# Training callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-7,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        f'{WORK_DIR}/best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    RawImageTestCallback(CLASSES, test_frequency=5)
]

print("Callbacks configured")

Callbacks configured


In [10]:
# PHASE 1: Train classifier head with class weights
print("="*60)
print("PHASE 1: Training classifier head")
print("="*60)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    class_weight=class_weight_dict,  # Force balanced learning
    verbose=1
)

print("\nPhase 1 training complete!")

PHASE 1: Training classifier head
Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2590 - loss: 3.0728
Epoch 1: val_accuracy improved from None to 0.25000, saving model to C:\Users\s62ht\Downloads\fish_model_efficientnet/best_model.keras

Epoch 1: finished saving model to C:\Users\s62ht\Downloads\fish_model_efficientnet/best_model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.2625 - loss: 2.9301 - val_accuracy: 0.2500 - val_loss: 2.3224 - learning_rate: 5.0000e-04
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 860ms/step - accuracy: 0.2741 - loss: 2.6820
Epoch 2: val_accuracy improved from 0.25000 to 0.25500, saving model to C:\Users\s62ht\Downloads\fish_model_efficientnet/best_model.keras

Epoch 2: finished saving model to C:\Users\s62ht\Downloads\fish_model_efficientnet/best_model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.2700 - loss: 2.6390 - val_accuracy: 0.2550 - val_loss: 2.2725 - learning_rate: 5.0000e-04
Epoch 3/30
100/100 ━━━━━━━

In [11]:
# Test model with raw images after phase 1
labels = [name for name, _ in sorted(train_generator.class_indices.items(), key=lambda x: x[1])]
phase1_accuracy = test_model_with_raw_images(model, labels, num_per_class=5)

if phase1_accuracy < 0.5:
    print("\n⚠️ WARNING: Phase 1 accuracy is below 50%. The model is not learning properly.")
    print("Consider:")
    print("  1. Checking if dataset is corrupted")
    print("  2. Increasing EPOCHS to 50")
    print("  3. Trying a different random seed")
else:
    print(f"\n✓ Phase 1 successful! Raw image accuracy: {phase1_accuracy*100:.1f}%")


Testing model with raw images...

Gilt-Head Bream:
  ✗ Shrimp (53.7%) [Gilt-Head Bream: 15.6%, Hourse Mackerel: 17.6%, Sea Bass: 13.0%, Shrimp: 53.7%]
  ✗ Shrimp (43.9%) [Gilt-Head Bream: 18.2%, Hourse Mackerel: 21.8%, Sea Bass: 16.1%, Shrimp: 43.9%]
  ✗ Shrimp (38.6%) [Gilt-Head Bream: 19.8%, Hourse Mackerel: 23.8%, Sea Bass: 17.8%, Shrimp: 38.6%]
  ✗ Shrimp (61.7%) [Gilt-Head Bream: 12.9%, Hourse Mackerel: 14.7%, Sea Bass: 10.7%, Shrimp: 61.7%]
  ✗ Shrimp (61.7%) [Gilt-Head Bream: 12.8%, Hourse Mackerel: 14.8%, Sea Bass: 10.7%, Shrimp: 61.7%]
  Class accuracy: 0/5 = 0.0%

Hourse Mackerel:
  ✗ Shrimp (60.8%) [Gilt-Head Bream: 13.1%, Hourse Mackerel: 15.1%, Sea Bass: 11.1%, Shrimp: 60.8%]
  ✗ Shrimp (58.7%) [Gilt-Head Bream: 13.6%, Hourse Mackerel: 16.0%, Sea Bass: 11.7%, Shrimp: 58.7%]
  ✗ Shrimp (60.8%) [Gilt-Head Bream: 13.1%, Hourse Mackerel: 15.1%, Sea Bass: 11.1%, Shrimp: 60.8%]
  ✗ Shrimp (56.1%) [Gilt-Head Bream: 14.4%, Hourse Mackerel: 17.0%, Sea Bass: 12.5%, Shrimp: 56.1%]
 

In [12]:
# PHASE 2: Fine-tune the entire model
print("\n" + "="*60)
print("PHASE 2: Fine-tuning entire model")
print("="*60)

# Unfreeze the base model
base_model.trainable = True

# Recompile with lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00005),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

print(f"Fine-tuning with {len(model.trainable_variables)} trainable variables\n")

# Train with fine-tuning
history_finetune = model.fit(
    train_generator,
    epochs=FINETUNE_EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("\nPhase 2 fine-tuning complete!")


PHASE 2: Fine-tuning entire model
Fine-tuning with 225 trainable variables

Epoch 1/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4148 - loss: 1.5731
Epoch 1: val_accuracy did not improve from 0.48875
100/100 ━━━━━━━━━━━━━━━━━━━━ 374s 3s/step - accuracy: 0.5003 - loss: 1.4739 - val_accuracy: 0.2500 - val_loss: 5.6842 - learning_rate: 5.0000e-05
Epoch 2/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7274 - loss: 1.2589
Epoch 2: val_accuracy did not improve from 0.48875
100/100 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.8031 - loss: 1.1862 - val_accuracy: 0.2450 - val_loss: 2.7223 - learning_rate: 5.0000e-05
Epoch 3/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9171 - loss: 1.0029
Epoch 3: val_accuracy improved from 0.48875 to 0.56000, saving model to C:\Users\s62ht\Downloads\fish_model_efficientnet/best_model.keras

Epoch 3: finished saving model to C:\Users\s62ht\Downloads\fish_model_efficientnet/best_model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 337s 3

In [13]:
# Final evaluation
print("\nEvaluating final model...")
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"\nFinal Validation Loss: {val_loss:.4f}")
print(f"Final Validation Accuracy: {val_accuracy:.4f}")


Evaluating final model...
25/25 ━━━━━━━━━━━━━━━━━━━━ 13s 528ms/step - accuracy: 1.0000 - loss: 0.6146

Final Validation Loss: 0.6146
Final Validation Accuracy: 1.0000


In [14]:
# Comprehensive test with raw images
final_accuracy = test_model_with_raw_images(model, labels, num_per_class=10)

if final_accuracy < 0.7:
    print("\n⚠️ WARNING: Final accuracy is below 70%.")
    print("The model may not work well in production.")
    print("\nRecommendations:")
    print("  1. Train for more epochs (try EPOCHS=50)")
    print("  2. Check dataset for mislabeled images")
    print("  3. Try different augmentation parameters")
else:
    print(f"\n✓ SUCCESS! Final raw image accuracy: {final_accuracy*100:.1f}%")
    print("Model is ready for deployment!")


Testing model with raw images...

Gilt-Head Bream:
  ✓ Gilt-Head Bream (95.9%) [Gilt-Head Bream: 95.9%, Hourse Mackerel: 1.7%, Sea Bass: 1.2%, Shrimp: 1.2%]
  ✓ Gilt-Head Bream (96.7%) [Gilt-Head Bream: 96.7%, Hourse Mackerel: 1.5%, Sea Bass: 0.8%, Shrimp: 0.9%]
  ✓ Gilt-Head Bream (97.3%) [Gilt-Head Bream: 97.3%, Hourse Mackerel: 1.2%, Sea Bass: 0.7%, Shrimp: 0.8%]
  ✓ Gilt-Head Bream (91.2%) [Gilt-Head Bream: 91.2%, Hourse Mackerel: 3.2%, Sea Bass: 3.3%, Shrimp: 2.2%]
  ✓ Gilt-Head Bream (95.5%) [Gilt-Head Bream: 95.5%, Hourse Mackerel: 1.9%, Sea Bass: 1.4%, Shrimp: 1.2%]
  ✓ Gilt-Head Bream (95.2%) [Gilt-Head Bream: 95.2%, Hourse Mackerel: 2.0%, Sea Bass: 1.6%, Shrimp: 1.2%]
  ✓ Gilt-Head Bream (96.8%) [Gilt-Head Bream: 96.8%, Hourse Mackerel: 1.3%, Sea Bass: 1.0%, Shrimp: 1.0%]
  ✓ Gilt-Head Bream (96.9%) [Gilt-Head Bream: 96.9%, Hourse Mackerel: 1.2%, Sea Bass: 1.0%, Shrimp: 0.9%]
  ✓ Gilt-Head Bream (97.4%) [Gilt-Head Bream: 97.4%, Hourse Mackerel: 1.0%, Sea Bass: 0.8%, Shrimp: 

In [15]:
# Save model in Keras format
keras_model_path = f'{WORK_DIR}/fish_model_efficientnet.keras'
model.save(keras_model_path)
print(f"Keras model saved to: {keras_model_path}")

Keras model saved to: C:\Users\s62ht\Downloads\fish_model_efficientnet/fish_model_efficientnet.keras


In [16]:
# Convert to TensorFlow Lite WITHOUT quantization
print("\nConverting to TensorFlow Lite (full precision)...")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# NO quantization - keeps full float32 precision
tflite_model = converter.convert()

# Save TFLite model
tflite_path = f'{WORK_DIR}/fish_classifier.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

file_size = os.path.getsize(tflite_path) / (1024 * 1024)
print(f"\nTFLite model saved: fish_classifier.tflite")
print(f"Size: {file_size:.2f} MB")


Converting to TensorFlow Lite (full precision)...
INFO:tensorflow:Assets written to: C:\Users\s62ht\AppData\Local\Temp\tmpqqtw7fmq\assets


INFO:tensorflow:Assets written to: C:\Users\s62ht\AppData\Local\Temp\tmpqqtw7fmq\assets


Saved artifact at 'C:\Users\s62ht\AppData\Local\Temp\tmpqqtw7fmq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_238')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2198583462608: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2198583462416: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2198551940560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2198551941712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2198551941328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2198551938832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2198551941520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2198551940944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2198551940752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2198551941136: TensorSpec(shape=(), dtype=tf.resour

In [17]:
# Generate labels.txt file
class_indices = train_generator.class_indices
sorted_classes = sorted(class_indices.items(), key=lambda x: x[1])
labels = [class_name for class_name, _ in sorted_classes]

labels_path = f'{WORK_DIR}/labels.txt'
with open(labels_path, 'w') as f:
    for label in labels:
        f.write(f"{label}\n")

print("\nLabels file created: labels.txt")
print("Contents:")
for i, label in enumerate(labels):
    print(f"  {i}: {label}")


Labels file created: labels.txt
Contents:
  0: Gilt-Head Bream
  1: Hourse Mackerel
  2: Sea Bass
  3: Shrimp


In [18]:
# Final test of TFLite model
print("\n" + "="*60)
print("FINAL TFLITE MODEL TEST")
print("="*60)

def preprocess_image_flutter_style(image_path):
    """Preprocess image exactly as Flutter does"""
    img = Image.open(image_path).convert('RGB')
    
    # Center crop to square
    width, height = img.size
    size = min(width, height)
    left = (width - size) // 2
    top = (height - size) // 2
    img = img.crop((left, top, left + size, top + size))
    
    # Resize to 224x224
    img = img.resize((224, 224), Image.BILINEAR)
    
    # Normalize: (pixel / 127.5) - 1.0
    img_array = np.array(img, dtype=np.float32)
    img_array = (img_array / 127.5) - 1.0
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

# Load TFLite interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Input shape: {input_details[0]['shape']}")
print(f"Output shape: {output_details[0]['shape']}")
print()

# Test 5 images from each class
total_correct = 0
total_tested = 0

for class_name in labels:
    class_dir = os.path.join(VAL_DIR, class_name)
    if not os.path.exists(class_dir):
        continue
    
    images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:5]
    
    print(f"Testing {class_name}:")
    class_correct = 0
    
    for img_name in images:
        img_path = os.path.join(class_dir, img_name)
        input_data = preprocess_image_flutter_style(img_path)
        
        # Run TFLite inference
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])[0]
        
        predicted_idx = np.argmax(output)
        predicted_label = labels[predicted_idx]
        confidence = output[predicted_idx]
        
        is_correct = predicted_label == class_name
        status = "✓" if is_correct else "✗"
        
        probs_str = ", ".join([f"{labels[i]}: {output[i]*100:.1f}%" for i in range(len(labels))])
        print(f"  {status} {img_name}: {predicted_label} ({confidence*100:.1f}%) [{probs_str}]")
        
        if is_correct:
            class_correct += 1
            total_correct += 1
        total_tested += 1
    
    print(f"  Class accuracy: {class_correct}/{len(images)} = {class_correct/len(images)*100:.1f}%")
    print()

tflite_accuracy = total_correct / total_tested if total_tested > 0 else 0
print("="*60)
print(f"TFLITE OVERALL ACCURACY: {total_correct}/{total_tested} = {tflite_accuracy*100:.1f}%")
print("="*60)


FINAL TFLITE MODEL TEST
Input shape: [  1 224 224   3]
Output shape: [1 4]

Testing Gilt-Head Bream:
  ✓ 00004.png: Gilt-Head Bream (95.9%) [Gilt-Head Bream: 95.9%, Hourse Mackerel: 1.7%, Sea Bass: 1.2%, Shrimp: 1.2%]


C:\Users\s62ht\AppData\Roaming\Python\Python313\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


  ✓ 00007.png: Gilt-Head Bream (96.7%) [Gilt-Head Bream: 96.7%, Hourse Mackerel: 1.5%, Sea Bass: 0.8%, Shrimp: 0.9%]
  ✓ 00012.png: Gilt-Head Bream (97.3%) [Gilt-Head Bream: 97.3%, Hourse Mackerel: 1.2%, Sea Bass: 0.7%, Shrimp: 0.8%]
  ✓ 00026.png: Gilt-Head Bream (91.2%) [Gilt-Head Bream: 91.2%, Hourse Mackerel: 3.2%, Sea Bass: 3.3%, Shrimp: 2.2%]
  ✓ 00028.png: Gilt-Head Bream (95.5%) [Gilt-Head Bream: 95.5%, Hourse Mackerel: 1.9%, Sea Bass: 1.4%, Shrimp: 1.2%]
  Class accuracy: 5/5 = 100.0%

Testing Hourse Mackerel:
  ✓ 00004.png: Hourse Mackerel (91.1%) [Gilt-Head Bream: 4.1%, Hourse Mackerel: 91.1%, Sea Bass: 2.4%, Shrimp: 2.4%]
  ✓ 00007.png: Hourse Mackerel (93.6%) [Gilt-Head Bream: 2.4%, Hourse Mackerel: 93.6%, Sea Bass: 2.1%, Shrimp: 1.9%]
  ✓ 00012.png: Hourse Mackerel (91.1%) [Gilt-Head Bream: 4.1%, Hourse Mackerel: 91.1%, Sea Bass: 2.4%, Shrimp: 2.4%]
  ✓ 00026.png: Hourse Mackerel (90.2%) [Gilt-Head Bream: 3.2%, Hourse Mackerel: 90.2%, Sea Bass: 3.8%, Shrimp: 2.8%]
  ✓ 000

In [19]:
# Final summary
print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"\nFiles created in: {WORK_DIR}")
print("  1. fish_classifier.tflite - Full precision model")
print("  2. labels.txt - Class labels")
print("  3. fish_model_efficientnet.keras - Keras model")
print("\nModel Performance:")
print(f"  Validation Accuracy: {val_accuracy:.2%}")
print(f"  Raw Image Accuracy: {final_accuracy:.2%}")
print(f"  TFLite Accuracy: {tflite_accuracy:.2%}")
print(f"  Model Size: {file_size:.2f} MB")

if tflite_accuracy >= 0.7:
    print("\n✅ SUCCESS! Model is ready for Flutter.")
    print("\nNext Steps:")
    print(f"  1. Copy {WORK_DIR}\\fish_classifier.tflite")
    print(f"     to c:\\Users\\s62ht\\Desktop\\Bahaar\\assets\\models\\")
    print(f"  2. Copy {WORK_DIR}\\labels.txt")
    print(f"     to c:\\Users\\s62ht\\Desktop\\Bahaar\\assets\\models\\")
    print("  3. Test in Flutter app")
else:
    print("\n❌ WARNING: Model accuracy is too low.")
    print("\nTroubleshooting:")
    print("  1. Increase EPOCHS to 50")
    print("  2. Check dataset for issues")
    print("  3. Try running training again with different random seed")

print("\n" + "="*60)


TRAINING COMPLETE

Files created in: C:\Users\s62ht\Downloads\fish_model_efficientnet
  1. fish_classifier.tflite - Full precision model
  2. labels.txt - Class labels
  3. fish_model_efficientnet.keras - Keras model

Model Performance:
  Validation Accuracy: 100.00%
  Raw Image Accuracy: 100.00%
  TFLite Accuracy: 100.00%
  Model Size: 18.42 MB

✅ SUCCESS! Model is ready for Flutter.

Next Steps:
  1. Copy C:\Users\s62ht\Downloads\fish_model_efficientnet\fish_classifier.tflite
     to c:\Users\s62ht\Desktop\Bahaar\assets\models\
  2. Copy C:\Users\s62ht\Downloads\fish_model_efficientnet\labels.txt
     to c:\Users\s62ht\Desktop\Bahaar\assets\models\
  3. Test in Flutter app

